In [1]:
import sys
import pandas as pd
import dateutil
import numpy as np
sys.path.append('../')
from utilities.era5_down import era5_down
import glob

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
date_parser = lambda x: dateutil.parser.parse(x, ignoretz=True)
# Improved function to sum dataframe columns which contain nan's
def nansumwrapper(a, **kwargs):
    if np.isnan(a).all():
        return np.nan
    else:
        return np.nansum(a, **kwargs)

In [4]:
#cs_file = '../data/SMB_bellin_diff_1.csv'
cs_file = '../data/SMB_fourc.csv'

dfmb = pd.read_csv(cs_file,
   delimiter='\t', parse_dates=['Date', 'Date1'], na_values='NAN')
labels = dfmb.drop_duplicates(subset=['Stake'])

In [5]:
t2m = 't2m'
rh2 = 'rh'
ff  = 'u2'
Prec = 'tp'
Snowfall = 'sf'
SWD = 'SWin'
LWD = 'LWin'
Pres = 'press'
fcc  = 'tcc'
msl  = 'msl'

In [6]:
files = sorted(glob.glob('../data/ERA_59_20_day/*.nc'))
len(files)

62

In [7]:
data_pro = []
for st in labels['Stake']:
    stake  = dfmb.loc[lambda df: df['Stake'] == st, :].copy().reset_index()
    df_day = era5_down(files, stake['Longitude'][0],
                       stake['Latitude'][0], 
                       stake['Elevation'][0])
    df_day['t2m_an'] = (df_day[t2m] - df_day[t2m].mean())/df_day[t2m].std()
    subset_f0 = []
    for i in range(len(stake)):
        subset = df_day.loc[str(stake.Date1[i]): str(stake.Date[i])].copy()
        subset['PDD'] = subset[t2m].where(subset[t2m] > 0, 0)
        subset['stake'] = stake['Stake'][0]
        subset_f = subset.groupby(['stake']).agg({t2m:np.mean, rh2:np.mean, ff:np.mean, 
                                         SWD:np.mean, LWD:np.mean, Prec:nansumwrapper,
                                         Snowfall:nansumwrapper, msl:np.mean,
                                         Pres:np.mean, fcc:np.mean, 't2m_an':np.mean,
                                         'PDD':nansumwrapper})
        subset_f.index = ([stake.Date[i]])
        subset_f0.append(subset_f)
    
    data_pro.append(pd.concat([stake, pd.concat(subset_f0).reset_index()], axis=1))

In [8]:
data_inp_four = pd.concat(data_pro)
data_inp_four.drop(['index', 'Date1', 'index'], axis=1, inplace=True)

In [9]:
data_inp_four.columns

Index(['Stake', 'Date', 'Latitude', 'Longitude', 'Elevation', 'Days', 'SMB',
       't2m', 'rh', 'u2', 'SWin', 'LWin', 'tp', 'sf', 'msl', 'press', 'tcc',
       't2m_an', 'PDD'],
      dtype='object')

In [10]:
data_inp_four

,Stake,Date,Latitude,Longitude,Elevation,Days,SMB,t2m,rh,u2,SWin,LWin,tp,sf,msl,press,tcc,t2m_an,PDD
0,PG01,2011-03-02,-62.201488,-58.583236,435.7,0,3.14,-0.842712,88.928459,14.502913,193.751175,262.329926,0.000240,0.000000,988.828979,938.897339,0.569043,1.283319,0.000000
1,PG01,2011-03-07,-62.201488,-58.583236,435.7,5,-3.03,-1.547089,91.561661,12.955721,103.991371,288.652130,0.027941,0.004176,984.745239,934.999451,0.884296,1.111962,0.000000
2,PG01,2011-03-09,-62.201488,-58.583236,435.7,2,0.10,-2.051198,93.916786,10.064571,71.619347,300.077026,0.010606,0.001503,986.691650,936.838501,0.940137,0.989325,0.000000
3,PG01,2013-02-11,-62.201488,-58.583236,435.7,705,0.05,-6.571172,87.573738,9.982855,101.991409,264.680542,1.763366,1.475362,990.608948,940.449097,0.847065,-0.110270,0.000000
4,PG01,2013-02-19,-62.201488,-58.583236,435.7,8,-0.08,-2.980143,88.867760,9.313358,137.897217,287.337769,0.024997,0.017200,981.617554,931.990601,0.897450,0.763336,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112,PG19,2016-02-17,-62.237672,-58.633624,96.8,44,-0.43,-0.284498,85.731506,9.542617,207.966919,274.654022,0.085414,0.065327,987.895020,976.440186,0.800041,0.676566,12.224274
113,PG19,2016-02-25,-62.237672,-58.633624,96.8,8,-0.20,0.197350,89.256035,8.389152,127.560768,287.325226,0.027218,0.014801,978.772095,967.441528,0.913182,0.796762,4.630981
114,PG19,2016-03-10,-62.237672,-58.633624,96.8,14,-0.45,0.442562,91.217651,9.195325,110.797836,284.019989,0.021845,0.006020,992.811584,981.320862,0.792833,0.857929,7.281006
115,PG19,2016-04-08,-62.237672,-58.633624,96.8,29,-0.48,-0.134649,85.319710,10.782786,70.121140,275.991150,0.125536,0.048219,985.026855,973.613281,0.811083,0.713946,15.088287


In [11]:
# reset SMB, tp, snowfall and PDD to 0 on 0 days observation
#data_inp_four[['SMB',Prec, Snowfall, 'PDD' ]] = data_inp_four[['SMB',Prec, Snowfall,
#                                                               'PDD' ]].where(data_inp_four['Days'] <= 100,
#                                                                              np.nan)
data_inp_four[['SMB',Prec, Snowfall, 'PDD' ]] = data_inp_four[['SMB',Prec, Snowfall,
                                                               'PDD' ]].where(data_inp_four['Days'] != 0,
                                                                              np.nan)

#data_inp_four[['PDD' ]] = data_inp_four[['PDD']].where(data_inp_four['SMB'] < 0, 0)
#data_inp_four[['PDD' ]] = data_inp_four[['PDD']].where(data_inp_four['Elevation'] < 260, 0) # four
#data_inp_four[['PDD' ]] = data_inp_four[['PDD']].where(data_inp_four['Elevation'] < 200, 0) # bell


data_inp_four.dropna(inplace=True)

In [12]:
data_inp_four

,Stake,Date,Latitude,Longitude,Elevation,Days,SMB,t2m,rh,u2,SWin,LWin,tp,sf,msl,press,tcc,t2m_an,PDD
1,PG01,2011-03-07,-62.201488,-58.583236,435.7,5,-3.03,-1.547089,91.561661,12.955721,103.991371,288.652130,0.027941,0.004176,984.745239,934.999451,0.884296,1.111962,0.000000
2,PG01,2011-03-09,-62.201488,-58.583236,435.7,2,0.10,-2.051198,93.916786,10.064571,71.619347,300.077026,0.010606,0.001503,986.691650,936.838501,0.940137,0.989325,0.000000
3,PG01,2013-02-11,-62.201488,-58.583236,435.7,705,0.05,-6.571172,87.573738,9.982855,101.991409,264.680542,1.763366,1.475362,990.608948,940.449097,0.847065,-0.110270,0.000000
4,PG01,2013-02-19,-62.201488,-58.583236,435.7,8,-0.08,-2.980143,88.867760,9.313358,137.897217,287.337769,0.024997,0.017200,981.617554,931.990601,0.897450,0.763336,0.000000
5,PG01,2013-02-27,-62.201488,-58.583236,435.7,8,-0.01,-3.258023,85.847710,9.903598,163.349258,263.866852,0.007287,0.003915,1001.603638,950.962769,0.673365,0.695735,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112,PG19,2016-02-17,-62.237672,-58.633624,96.8,44,-0.43,-0.284498,85.731506,9.542617,207.966919,274.654022,0.085414,0.065327,987.895020,976.440186,0.800041,0.676566,12.224274
113,PG19,2016-02-25,-62.237672,-58.633624,96.8,8,-0.20,0.197350,89.256035,8.389152,127.560768,287.325226,0.027218,0.014801,978.772095,967.441528,0.913182,0.796762,4.630981
114,PG19,2016-03-10,-62.237672,-58.633624,96.8,14,-0.45,0.442562,91.217651,9.195325,110.797836,284.019989,0.021845,0.006020,992.811584,981.320862,0.792833,0.857929,7.281006
115,PG19,2016-04-08,-62.237672,-58.633624,96.8,29,-0.48,-0.134649,85.319710,10.782786,70.121140,275.991150,0.125536,0.048219,985.026855,973.613281,0.811083,0.713946,15.088287


In [13]:
#data_inp_four.to_csv('../data/SMB_input_belli_ERA5.csv', index=False, sep='\t')
data_inp_four.to_csv('../data/SMB_input_four_ERA5.csv', index=False, sep='\t')